# Generating music with RNN's the simple way

Whilst most of the RNN applications required large amount of training time and resources, music generation is surprisingly easy to achieve *good* results. In this tutorial heavily based on \cite{blog} we will apply simple RNN's networks to produce traditional folk tunes.

## The data

Perhaps the most important question we must answer before getting in greater detail is which data representation we will use, since this will determine the complexity of the model required. Here, since we don't want to waste weeks training our models we will use a very simple approach: The ABC notation.

### ABC notation

Citing from the wikipedia \cite{wikipedia}:

>ABC notation is a shorthand form of musical notation. In basic form it uses the letters A through G to represent the given notes, with other elements used to place added value on these - sharp, flat, the length of the note, key, ornamentation. Lines in the first part of the tune notation, beginning with a letter followed by a colon, indicate various aspects of the tune such as the index, when there are more than one tune in a file (X:), the title (T:), the time signature (M:), the default note length (L:), the type of tune (R:) and the key (K:). Lines following the key designation represent the tune. It can be translated into traditional music notation using one of the abc conversion tools.

For instance here we have an example of "Greensleves" written in ABC notation

``` abc
X:870
T:Greensleeves
C:anon.
O:England
R:Broadside ballad
Z:Transcribed by Frank Nordberg - http://www.musicaviva.com
F:http://abc.musicaviva.com/tunes/england/greensleeves-dorian.abc
M:6/4
L:1/4
Q:1/2=110
K:Gdor
G|"Gm"B2c d>ed|"F"c2A F>GA|"Gm"B2A G>^FG|"Dm"A2^F D2G|
"Gm"B2c d>ed|"F"c2A F>GA|"Gm"B>AG "D"^F>EF|"Gm"G3 G2z|
"Bb"f3 f>ed|"F"c2A "Dm"F>GA|"Gm"B2G G>^FG|"Dm"A2^F D2z|
"Bb"f3 f>ed|"F"c2A "Dm"F>GA|"Gm"B>AG "D"^F>EF|"G"G3 G2|]
```

This notation is great because it enables to generate music only with the common alphabet set, without complicated notations and symbols. Also, a vast amount of tunes is available in the internets, which is important for learning.

## The model

For this exercise we will be using a simple implementation of rnn for text called char-rnn, originally developed by the omnipresent Karpathy. An implementation written in tersorflow is available at github.

It has two scripts, one for training and one for sampling. We can see the many options available calling them with *-h*.

In [ ]:
!git clone https://github.com/sherjilozair/char-rnn-tensorflow.git
!python ./char-rnn-tensorflow/train.py -h
!python ./char-rnn-tensorflow/sample.py -h

With regard to the hyperparameters by default, if we take a look into train.py we can see which ones it uses.


In [ ]:
 parser.add_argument('--data_dir', type=str, default='data/tinyshakespeare',
                       help='data directory containing input.txt')
    parser.add_argument('--save_dir', type=str, default='save',
                       help='directory to store checkpointed models')
    parser.add_argument('--rnn_size', type=int, default=128,
                       help='size of RNN hidden state')
    parser.add_argument('--num_layers', type=int, default=2,
                       help='number of layers in the RNN')
    parser.add_argument('--model', type=str, default='lstm',
                       help='rnn, gru, or lstm')
    parser.add_argument('--batch_size', type=int, default=50,
                       help='minibatch size')
    parser.add_argument('--seq_length', type=int, default=50,
                       help='RNN sequence length')
    parser.add_argument('--num_epochs', type=int, default=50,
                       help='number of epochs')
    parser.add_argument('--save_every', type=int, default=1000,
                       help='save frequency')
    parser.add_argument('--grad_clip', type=float, default=5.,
                       help='clip gradients at this value')
    parser.add_argument('--learning_rate', type=float, default=0.002,
                       help='learning rate')
    parser.add_argument('--decay_rate', type=float, default=0.97,
                       help='decay rate for rmsprop')                       
    parser.add_argument('--init_from', type=str, default=None,
                       help="""continue training from saved model at this path. Path must contain files saved by previous training process: 
                            'config.pkl'        : configuration;
                            'chars_vocab.pkl'   : vocabulary definitions;
                            'checkpoint'        : paths to model file(s) (created by tf).
                                                  Note: this file contains absolute paths, be careful when moving files around;
                            'model.ckpt-*'      : file(s) with model definition (created by tf)
                        """)

Notice how we can especify the size of the hidden state, the number of layers, the number of steps used for training and the typical stuff related to SGD, batch size, learning rate and others. Those parameters are used later to build the network inside model.py. The relevant part is shown commented below.


In [ ]:
# First we chose the type of RNN according the parameters provided. The default value is lstm, yet we will use gru.
if args.model == 'rnn':
    cell_fn = rnn_cell.BasicRNNCell
elif args.model == 'gru':
    cell_fn = rnn_cell.GRUCell
elif args.model == 'lstm':
    cell_fn = rnn_cell.BasicLSTMCell
else:
    raise Exception("model type not supported: {}".format(args.model))

# The cell is initialied and replicated as layers have been determined. This value is 128 x 2 layers in the defaults
cell = cell_fn(args.rnn_size)

self.cell = cell = rnn_cell.MultiRNNCell([cell] * args.num_layers)

# The placeholders of the network are initialized, along with the initial state.
self.input_data = tf.placeholder(tf.int32, [args.batch_size, args.seq_length])
self.targets = tf.placeholder(tf.int32, [args.batch_size, args.seq_length])
self.initial_state = cell.zero_state(args.batch_size, tf.float32)

# The output softmax weights and biases are built here
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [args.rnn_size, args.vocab_size])
    softmax_b = tf.get_variable("softmax_b", [args.vocab_size])
    with tf.device("/cpu:0"):
        # The embedding is created here, mapping the inputs to the state of the rnn
        embedding = tf.get_variable("embedding", [args.vocab_size, args.rnn_size])
        inputs = tf.split(1, args.seq_length, tf.nn.embedding_lookup(embedding, self.input_data))
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

# The loop function passes does a forward pass of the data through the RNN cells        
def loop(prev, _):
    prev = tf.matmul(prev, softmax_w) + softmax_b
    prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
    return tf.nn.embedding_lookup(embedding, prev_symbol)

# We get the final output of the network
outputs, last_state = seq2seq.rnn_decoder(inputs, self.initial_state, cell, loop_function=loop if infer else None, scope='rnnlm')
output = tf.reshape(tf.concat(1, outputs), [-1, args.rnn_size])
# We input the output of the cell to the softmax
self.logits = tf.matmul(output, softmax_w) + softmax_b
self.probs = tf.nn.softmax(self.logits)
# We set the loss
loss = seq2seq.sequence_loss_by_example([self.logits],
        [tf.reshape(self.targets, [-1])],
        [tf.ones([args.batch_size * args.seq_length])],
        args.vocab_size)
# The loss is averaged
self.cost = tf.reduce_sum(loss) / args.batch_size / args.seq_length
self.final_state = last_state
# Learning rate variable
self.lr = tf.Variable(0.0, trainable=False)
tvars = tf.trainable_variables()
# Gradient clipping
grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),
        args.grad_clip)
# Optimize using Adam
optimizer = tf.train.AdamOptimizer(self.lr)
self.train_op = optimizer.apply_gradients(zip(grads, tvars))

Since we don't want to mess too much with the hyperparameters, we will take the default ones as good enough, with the exception of using GRU instead of LTSM since the amount of data is not large. First we need the data. We will use this (http://www.norbeck.nu/abc/hn201602.zip), but any abc tune collection is equally good. After concat all the files in one we are ready to train. Further cleaning is possible, and it will improve the resulting generator.

In [ ]:
!wget -N http://www.norbeck.nu/abc/hn201602.zip
!unzip hn201602.zip
!mv ./s/* ./i
!cat ./i/*abc > ./char-rnn-tensorflow/data/input.txt
!tail -n 100 ./char-rnn-tensorflow/data/input.txt
!ls ./char-rnn-tensorflow/data
!rm -R ./s ./i

# Training

We are ready to start training.

In [ ]:
!python char-rnn-tensorflow/train.py --model gru --data_dir ./char-rnn-tensorflow/data --save_dir ./char-rnn-tensorflow/save